In [7]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 1s (183 kB/s)
Reading package li

In [8]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-11 18:40:39--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  4.84MB/s    in 0.2s    

2022-09-11 18:40:40 (4.84 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [10]:
from pyspark import SparkFiles
url = "https://hhpp-bucket.s3.amazonaws.com/HoustonSuperNeighborhoods.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("HoustonSuperNeighborhoods.csv"), sep=",", header=True, inferSchema=True)
df.show()

+---+--------------------+----------------+-----------------------+
| id|                name|total_population|median_household_income|
+---+--------------------+----------------+-----------------------+
|  1|    Kashmere Gardens|         10055.0|                23102.0|
|  2|Trinity / Houston...|         15789.0|                27789.0|
|  3|           Settegast|          2981.0|                28821.0|
|  4|Denver Harbor / P...|         17571.0|                29273.0|
|  5|          South Park|         21208.0|                29475.0|
|  6|            Westwood|         19530.0|                29527.0|
|  7|  Greater Fifth Ward|         19687.0|                30535.0|
|  8|Clinton Park / Tr...|          3140.0|                31826.0|
|  9|        East Houston|         18580.0|                31826.0|
| 10|       Magnolia Park|         16999.0|                32039.0|
| 11|Gulfgate Rivervie...|         12723.0|                32595.0|
| 12|       Pleasantville|          2860.0|     

In [14]:
# Store environmental variable

from getpass import getpass
password = getpass('Enter database password')

# Configure settings for RDS

mode = "append"
jdbc_url="jdbc:postgresql://hhpp-db.cib8i0dtf6rx.us-east-1.rds.amazonaws.com:5432/hhppdb"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [15]:
# Write df to table in RDS
df.write.jdbc(url=jdbc_url, table='houston_super_neighborhoods', mode=mode, properties=config)

In [17]:
# Read from table
read_df = spark.read.jdbc(url=jdbc_url, table='houston_super_neighborhoods', properties=config)
read_df.show()

+---+--------------------+----------------+-----------------------+
| id|                name|total_population|median_household_income|
+---+--------------------+----------------+-----------------------+
|  1|    Kashmere Gardens|           10055|                  23102|
|  2|Trinity / Houston...|           15789|                  27789|
|  3|           Settegast|            2981|                  28821|
|  4|Denver Harbor / P...|           17571|                  29273|
|  5|          South Park|           21208|                  29475|
|  6|            Westwood|           19530|                  29527|
|  7|  Greater Fifth Ward|           19687|                  30535|
|  8|Clinton Park / Tr...|            3140|                  31826|
|  9|        East Houston|           18580|                  31826|
| 10|       Magnolia Park|           16999|                  32039|
| 11|Gulfgate Rivervie...|           12723|                  32595|
| 12|       Pleasantville|            2860|     